## Importing libraries

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Table, Column, Float, Integer, MetaData
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show

##Loading data

In [2]:
ideal_df = pd.read_csv('ideal.csv')
test_df = pd.read_csv('test.csv')
train_df = pd.read_csv('train.csv')

print("Ideal data: \n", ideal_df.head())
print("\n Test data: \n", test_df.head())
print("\n Train data: \n", train_df.head())

Ideal data: 
       x        y1        y2        y3        y4        y5        y6        y7  \
0 -20.0 -0.912945  0.408082  9.087055  5.408082 -9.087055  0.912945 -0.839071   
1 -19.9 -0.867644  0.497186  9.132356  5.497186 -9.132356  0.867644 -0.865213   
2 -19.8 -0.813674  0.581322  9.186326  5.581322 -9.186326  0.813674 -0.889191   
3 -19.7 -0.751573  0.659649  9.248426  5.659649 -9.248426  0.751573 -0.910947   
4 -19.6 -0.681964  0.731386  9.318036  5.731386 -9.318036  0.681964 -0.930426   

         y8        y9  ...        y41        y42       y43       y44  \
0 -0.850919  0.816164  ... -40.456474  40.204040  2.995732 -0.008333   
1  0.168518  0.994372  ... -40.233820  40.048590  2.990720 -0.008340   
2  0.612391  1.162644  ... -40.006836  39.890660  2.985682 -0.008347   
3 -0.994669  1.319299  ... -39.775787  39.729824  2.980619 -0.008354   
4  0.774356  1.462772  ... -39.540980  39.565693  2.975530 -0.008361   

         y45       y46       y47       y48       y49       y50  
0

##Loading files to database

In [3]:
# Create a sqlite engine
engine = create_engine('sqlite:///mydatabase.db', echo=False)

# Load the dataframes into the database
ideal_df.to_sql('ideal_table', con=engine, if_exists='replace', index=False)
test_df.to_sql('test_table', con=engine, if_exists='replace', index=False)
train_df.to_sql('train_table', con=engine, if_exists='replace', index=False)

print("Data successfully loaded into the database.")

Data successfully loaded into the database.


In [4]:
import sqlite3

# Connect to the SQLite database to list all tables
conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()

# Query all table names in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in the database:", tables)

# Check if the 'train_table' exists
cursor.execute("PRAGMA table_info(train_table);")
columns = cursor.fetchall()
print("Columns in 'train_table':", columns)

# Close the connection
conn.close()


Tables in the database: [('ideal_table',), ('test_table',), ('train_table',)]
Columns in 'train_table': [(0, 'x', 'FLOAT', 0, None, 0), (1, 'y1', 'FLOAT', 0, None, 0), (2, 'y2', 'FLOAT', 0, None, 0), (3, 'y3', 'FLOAT', 0, None, 0), (4, 'y4', 'FLOAT', 0, None, 0)]


## Identifying best fitting ideal functions

In [5]:
# Function to find the best fitting ideal functions for each training function
def find_best_fit_ideal_functions(train_data, ideal_data):
    """
    Finds the best fitting ideal function for each training function by calculating
    the mean squared error between each training column and each ideal column.
    
    Args:
        train_data (pd.DataFrame): Training data with columns 'x', 'y1', 'y2', etc.
        ideal_data (pd.DataFrame): Ideal data with columns 'x', 'y1', 'y2', etc.
        
    Returns:
        pd.DataFrame: A DataFrame mapping each training function to its best fitting ideal function.
    """
    # Get the column names for y values in both datasets
    train_y_cols = [col for col in train_data.columns if col.startswith('y')]
    ideal_y_cols = [col for col in ideal_data.columns if col.startswith('y')]
    
    # Create a DataFrame to store the results
    best_fits = pd.DataFrame(columns=['train_function', 'ideal_function', 'mse'])
    
    # For each training function, find the best fitting ideal function
    for train_col in train_y_cols:
        train_y = train_data[train_col].values
        
        # Calculate MSE for each ideal function
        mse_values = []
        for ideal_col in ideal_y_cols:
            ideal_y = ideal_data[ideal_col].values
            
            # Calculate mean squared error
            mse = np.mean((train_y - ideal_y) ** 2)
            mse_values.append((ideal_col, mse))
        
        # Find the ideal function with the minimum MSE
        best_ideal_col, min_mse = min(mse_values, key=lambda x: x[1])
        
        # Add to results DataFrame
        best_fits = best_fits._append({
            'train_function': train_col,
            'ideal_function': best_ideal_col,
            'mse': min_mse
        }, ignore_index=True)
    
    return best_fits

# Function to load data from SQLite database
def load_data_from_db(db_url, table_name):
    """
    Loads data from the SQLite database into a pandas DataFrame.

    Args:
        db_url (str): The URL for the SQLite database.
        table_name (str): The table name to load.

    Returns:
        DataFrame: The loaded dataset.
    """
    engine = create_engine(db_url)

    # Verify if the table exists in the database before loading
    query = f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}';"
    result = pd.read_sql(query, engine)

    if result.empty:
        raise ValueError(f"Table '{table_name}' not found in the database.")

    return pd.read_sql(f"SELECT * FROM {table_name}", engine)

# Example usage:
if __name__ == "__main__":
    try:
        # Load the training and ideal datasets
        train_data = load_data_from_db("sqlite:///mydatabase.db", "train_table")
        ideal_data = load_data_from_db("sqlite:///mydatabase.db", "ideal_table")

        # Find the best fitting ideal functions for each training function
        best_ideal_df = find_best_fit_ideal_functions(train_data, ideal_data)

        # Output or further process the results (e.g., visualization, saving)
        print("\n",best_ideal_df)

    except Exception as e:
        print(f"An error occurred: {e}")


   train_function ideal_function       mse
0             y1            y42  0.085616
1             y2            y41  0.089005
2             y3            y11  0.074655
3             y4            y48  0.079909


C:\Users\dhruv\AppData\Local\Temp\ipykernel_30728\544162565.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  best_fits = best_fits._append({


## Mapping test data to ideal functions.

In [6]:
def map_test_data_to_ideal(test_df, best_ideal_df):
    """
    Maps the test data to the ideal functions based on the best fit.
    Replace this with your actual logic to map test data to ideal functions.

    Args:
        test_df (pd.DataFrame): Test data.
        best_ideal_df (pd.DataFrame): DataFrame of the best fitting ideal functions for each training function

    Returns:
        pd.DataFrame: A DataFrame containing the mapped test data.
    """
    mapped_test_data = test_df.copy()
    # Add a column to indicate the assigned ideal function. In real-world scenarios, this would be determined by a more sophisticated method. Instead of directly assigning best_ideal_df.index, create a list of ideal function indices that repeats to match the length of the test_df
    mapped_test_data['ideal_function'] = np.tile(best_ideal_df.index, len(test_df) // len(best_ideal_df.index) + 1)[:len(test_df)]
    return mapped_test_data

# Example usage (assuming 'best_ideal_df' is already computed)
mapped_test_data = map_test_data_to_ideal(test_df, best_ideal_df)
print("\nMapped Test Data:\n", mapped_test_data)


Mapped Test Data:
        x          y  ideal_function
0   17.5  34.161040               0
1    0.3   1.215102               1
2   -8.7 -16.843908               2
3  -19.2 -37.170870               3
4  -11.0 -20.263054               0
..   ...        ...             ...
95  -1.9  -4.036904               3
96  12.2  -0.010358               0
97  16.5 -33.964134               1
98   5.3 -10.291622               2
99  17.9  28.078455               3

[100 rows x 3 columns]


##Visualizing data

In [7]:
# Visualize the mapping of test data to ideal functions
def visualize_mapping(test_df, ideal_df, mapped_test_data):
    """
    Visualizes the mapping of test data to ideal functions.
    
    Args:
        test_df (pd.DataFrame): Test data.
        ideal_df (pd.DataFrame): Ideal data.
        mapped_test_data (pd.DataFrame): Mapped test data with ideal function assignments.
    """
    # Create a Bokeh figure
    p = figure(title="Test Data Mapped to Ideal Functions", 
               x_axis_label='x', 
               y_axis_label='y',
               width=800, 
               height=600)
    
    # Get unique ideal functions
    unique_ideal_functions = mapped_test_data['ideal_function'].unique()
    
    # Define colors for different ideal functions
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'black', 'cyan']
    
    # Plot test data points colored by their assigned ideal function
    for i, ideal_func in enumerate(unique_ideal_functions):
        subset = mapped_test_data[mapped_test_data['ideal_function'] == ideal_func]
        color = colors[i % len(colors)]
        
        # Plot test data points
        p.circle(subset['x'], subset['y'], size=8, color=color, alpha=0.7, 
                 legend_label=f"Test Data (Ideal {ideal_func})")
        
        # Plot corresponding ideal function line
        if isinstance(ideal_func, str) and ideal_func.startswith('y'):
            p.line(ideal_df['x'], ideal_df[ideal_func], line_width=2, color=color, 
                   alpha=0.5, legend_label=f"Ideal {ideal_func}")
    
    # Configure legend
    p.legend.location = "top_left"
    p.legend.click_policy = "hide"
    
    # Show the plot
    show(p)

# Example usage (assuming all required variables are defined)
visualize_mapping(test_df, ideal_df, mapped_test_data)